In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_horizontal_wavelet_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,46520.75,0.85,0.22,0.15,0.01,0.00,-0.00,-0.00
3,0.85,9282.43,-0.09,0.05,-0.00,0.00,-0.00,0.00
4,0.22,-0.09,1755.89,0.01,0.00,-0.00,0.00,-0.00
5,0.15,0.05,0.01,275.64,0.00,0.00,0.00,-0.00
6,0.01,-0.00,0.00,0.00,40.10,0.00,0.00,0.00
7,0.00,0.00,-0.00,0.00,0.00,5.36,-0.00,0.00
8,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.69,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.08


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00004,0.00002,0.00004,0.00000,0.00000,-0.00000,-0.00001
3,0.00004,1.00000,-0.00002,0.00003,-0.00000,0.00001,-0.00003,0.00003
4,0.00002,-0.00002,1.00000,0.00002,0.00001,-0.00004,0.00004,-0.00002
5,0.00004,0.00003,0.00002,1.00000,0.00002,0.00001,0.00001,-0.00002
6,0.00000,-0.00000,0.00001,0.00002,1.00000,0.00002,0.00002,0.00002
7,0.00000,0.00001,-0.00004,0.00001,0.00002,1.00000,-0.00003,0.00001
8,-0.00000,-0.00003,0.00004,0.00001,0.00002,-0.00003,1.00000,0.00002
9,-0.00001,0.00003,-0.00002,-0.00002,0.00002,0.00001,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

1.2689769825923771

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.70756846e+08 1.05530379e+07 3.67029269e+05 8.68627771e+03
 1.72379088e+02 2.73935933e+00 3.40815289e-02 6.54184353e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999547,-0.029618,-0.005356,-0.000837,-0.000122,-0.000016,-0.000002,-2.409691e-07
1,0.029422,0.999015,-0.032828,-0.004936,-0.000718,-0.000096,-0.000013,-1.257008e-06
2,0.006287,0.032477,0.998920,-0.032316,-0.004565,-0.000612,-0.000078,-8.792533e-06
3,0.001179,0.005922,0.031938,0.998766,-0.037238,-0.004845,-0.000622,-6.894869e-05
4,0.000214,0.001076,0.005688,0.036806,0.998257,-0.045408,-0.005693,-6.275382e-04
5,0.000038,0.000191,0.001012,0.006429,0.044723,0.996823,-0.065208,-7.017226e-03
6,0.000007,0.000037,0.000192,0.001227,0.008439,0.063717,0.991812,-1.103463e-01
7,0.000002,0.000008,0.000043,0.000274,0.001880,0.014083,0.109654,9.938682e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0004534020118236448,
 0.0009845965589359018,
 0.0010804934554750067,
 0.0012342652845226398,
 0.0017429956176561179,
 0.003177022205367974,
 0.008187988781017452,
 0.006131773674958763]